In [1]:
# import keras
# import tensorflow as tf
from keras import layers, optimizers
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Reshape, TimeDistributed, Input, LSTM
from keras.utils import np_utils, plot_model
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg19 import VGG19

Using TensorFlow backend.


In [2]:
batch_size = 64
window_size = 100 # 64 or 32 length LSTM

In [ ]:
def model_functional():
    input_lmd = Input(shape=(20, 64, 1)) # grayscale image = (20x64)
    input_shape = Input(shape=(window_size, 20, 64, 1))
    
    # build the convolutional block
    conv_first1 = Conv2D(32, (1, 3), padding='same')(input_lmd)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = Conv2D(32, (3, 1))(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = MaxPooling2D((1, 4), strides=(1, 4), padding='same')(conv_first1) # 18x16

    conv_first1 = Conv2D(32, (1, 3), padding='same')(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = Conv2D(32, (3, 1))(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = MaxPooling2D((1, 4), strides=(1, 4), padding='same')(conv_first1) # 16x4

    conv_first1 = Conv2D(32, (1, 5), padding='same')(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = Conv2D(32, (5, 1))(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = MaxPooling2D((1, 4), strides=(1, 4), padding='same')(conv_first1) # 12x1
    
    # build the inception module
    convsecond_1 = Conv2D(32, (1, 1), padding='same')(conv_first1)
    convsecond_1 = keras.layers.LeakyReLU(alpha=0.01)(convsecond_1)
    convsecond_1 = Conv2D(32, (3, 1), padding='same')(convsecond_1)
    convsecond_1 = keras.layers.LeakyReLU(alpha=0.01)(convsecond_1)

    convsecond_2 = Conv2D(32, (1, 1), padding='same')(conv_first1)
    convsecond_2 = keras.layers.LeakyReLU(alpha=0.01)(convsecond_2)
    convsecond_2 = Conv2D(32, (5, 1), padding='same')(convsecond_2)
    convsecond_2 = keras.layers.LeakyReLU(alpha=0.01)(convsecond_2)

    convsecond_3 = MaxPooling2D((3, 1), strides=(1, 1), padding='same')(conv_first1)
    convsecond_3 = Conv2D(32, (1, 1), padding='same')(convsecond_3)
    convsecond_3 = keras.layers.LeakyReLU(alpha=0.01)(convsecond_3)
    
    convsecond_output = keras.layers.concatenate([convsecond_1, convsecond_2, convsecond_3], axis=3) # axis=3 for filter axis

    # Flattern output to feed 1D vector to LSTM
    conv_flat = Flatten()(convsecond_output)
    
    # CNN model
    model_cnn = Model(inputs=input_lmd, outputs=conv_flat)

    # time distributed layer
    processed_sequences = TimeDistributed(model_cnn)(input_shape)
    
    # build the last LSTM layer
    conv_lstm = LSTM(32)(processed_sequences)

    # build the output layer
    out = Dense(3, activation='softmax')(conv_lstm)
    model_cnn_lstm = Model(inputs=input_shape, outputs=out)
    adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1)
    model_cnn_lstm.compile(optimizer=adam, loss='categorical_crossentropy', metrics=[keras.metrics.Accuracy(),
                                                                            keras.metrics.CategoricalAccuracy(),
                                                                            keras.metrics.CategoricalCrossentropy(),
                                                                            keras.metrics.Precision(),
                                                                            keras.metrics.Recall()])
    return model_cnn, model_cnn_lstm

model_cnn, cnn_lob_model = model_functional()
# model_cnn.summary()

In [ ]:
def model_InceptionV3_extractor():
    # create the base pre-trained model
    base_model = InceptionV3(weights='imagenet', include_top=False)

    # add a global spatial average pooling layer
    x = base_model.output
    # first: train only the top layers (which were randomly initialized)
    # i.e. freeze all convolutional InceptionV3 layers
    for layer in base_model.layers:
        layer.trainable = False

    # let's add a fully-connected layer
    inception_model = Dense(1024, activation='relu')(x)

    # this is the model we will train
    model_cnn = Model(inputs=base_model.input, outputs=inception_model)

    # time distributed layer
    input_shape = Input(shape=(window_size, 20, 64, 1))
    processed_sequences = TimeDistributed(model_cnn)(input_shape)
    
    # build the last LSTM layer
    conv_lstm = LSTM(64)(processed_sequences)

    # build the output layer
    out = Dense(3, activation='softmax')(conv_lstm)
    model_cnn_lstm = Model(inputs=input_shape, outputs=out)
    model_cnn_lstm.compile(optimizer=SGD(lr=0.0001, momentum=0.9),
                           loss='categorical_crossentropy', metrics=[keras.metrics.Accuracy(),
                                                                            keras.metrics.CategoricalAccuracy(),
                                                                            keras.metrics.CategoricalCrossentropy(),
                                                                            keras.metrics.Precision(),
                                                                            keras.metrics.Recall()])
    return model_cnn, model_cnn_lstm

In [ ]:
def model_VGG19_extractor():
    # create the base pre-trained model
    base_model = VGG19(weights='imagenet', include_top=False)

    # add a global spatial average pooling layer
    x = base_model.output
    # first: train only the top layers (which were randomly initialized)
    # i.e. freeze all convolutional InceptionV3 layers
    for layer in base_model.layers:
        layer.trainable = False

    # let's add a fully-connected layer
    vgg_model = Dense(1024, activation='relu')(x)

    # this is the model we will train
    model_cnn = Model(inputs=base_model.input, outputs=vgg_model) # outputs=base_model.get_layer('block4_pool').output

    # time distributed layer
    input_shape = Input(shape=(window_size, 20, 64, 1))
    processed_sequences = TimeDistributed(model_cnn)(input_shape)
    
    # build the last LSTM layer
    conv_lstm = LSTM(64)(processed_sequences)

    # build the output layer
    out = Dense(3, activation='softmax')(conv_lstm)
    model_cnn_lstm = Model(inputs=input_shape, outputs=out)
    model_cnn_lstm.compile(optimizer=SGD(lr=0.0001, momentum=0.9),
                           loss='categorical_crossentropy', metrics=[keras.metrics.Accuracy(),
                                                                            keras.metrics.CategoricalAccuracy(),
                                                                            keras.metrics.CategoricalCrossentropy(),
                                                                            keras.metrics.Precision(),
                                                                            keras.metrics.Recall()])
    return model_cnn, model_cnn_lstm